# 🌿 🧱 In Depth: Matrix Decomposition Techniques

Matrix decomposition is a foundational concept in linear algebra and is extensively used in fields such as machine learning, data science, and numerical analysis. It allows us to break down a complex matrix into simpler, more manageable components. This notebook will cover two primary matrix decomposition techniques:

1. **Eigen Decomposition**: Used for square matrices.
2. **Singular Value Decomposition (SVD)**: Used for any matrix, square or non-square.

These techniques help us understand the structure of matrices and have numerous practical applications, such as Principal Component Analysis (PCA), image compression, and more.


# Part 1: Eigen Decomposition for Square Matrices

**Eigen Decomposition** is a process of factorizing a square matrix into its eigenvalues and eigenvectors. Eigenvalues and eigenvectors are fundamental concepts in linear algebra and provide significant insights into the properties of a matrix.

### Explanation:

For a given square matrix $A$, if there exists a non-zero vector $v$ and a scalar $\lambda$ such that:

$$ A \times v = \lambda \times v $$

Then:
- $v$ is called an **eigenvector** of the matrix $A$.
- $\lambda$ is called the corresponding **eigenvalue**.

The matrix $A$ can be decomposed into the following form:

$$ A = V \times \Lambda \times V^{-1} $$

Where:
- $V$ is the matrix of eigenvectors.
- $\Lambda$ is a diagonal matrix containing the eigenvalues.
- $V^{-1}$ is the inverse of the matrix of eigenvectors.

This decomposition shows that the matrix $A$ can be represented as a product of its eigenvectors and eigenvalues.

### Proof (Sketch):

The eigenvalue equation can be rewritten as:

$$ (A - \lambda I) v = 0 $$

Where $I$ is the identity matrix of the same dimension as $A$. For a non-trivial solution to exist, the determinant of $(A - \lambda I)$ must be zero:

$$ \text{det}(A - \lambda I) = 0 $$

This is known as the **characteristic equation**, and solving it yields the eigenvalues $\lambda$. Once the eigenvalues are found, the corresponding eigenvectors can be determined by solving the system of linear equations $(A - \lambda I) v = 0$ for each eigenvalue $\lambda$.

Eigen decomposition is only possible for square matrices, and the matrix must have enough linearly independent eigenvectors to form the matrix $V$.


## From scratch:

In [1]:
import numpy as np

def power_iteration(A, num_iterations=1000, tolerance=1e-9):
    """
    Perform power iteration to find the dominant eigenvalue and eigenvector of matrix A.
    
    Parameters:
    - A: The input square matrix.
    - num_iterations: Number of iterations to run the power iteration algorithm.
    - tolerance: The convergence tolerance for stopping the iteration.
    
    Returns:
    - eigenvalue: The dominant eigenvalue of matrix A.
    - eigenvector: The corresponding eigenvector of matrix A.
    """
    # Step 1: Initialize a random vector
    n = A.shape[1]
    b_k = np.random.rand(n)
    
    # Step 2: Perform the iteration
    for _ in range(num_iterations):
        # Matrix-vector multiplication
        b_k1 = np.dot(A, b_k)
        
        # Normalize the vector
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm
        
        # Calculate the Rayleigh quotient for the dominant eigenvalue
        eigenvalue = np.dot(b_k.T, np.dot(A, b_k)) / np.dot(b_k.T, b_k)
        
        # Check for convergence
        if np.linalg.norm(np.dot(A, b_k) - eigenvalue * b_k) < tolerance:
            break

    return eigenvalue, b_k

def deflate_matrix(A, eigenvalue, eigenvector):
    """
    Deflate the matrix by subtracting the outer product of the found eigenvector.
    
    Parameters:
    - A: The original matrix.
    - eigenvalue: The found eigenvalue.
    - eigenvector: The corresponding eigenvector.
    
    Returns:
    - A_deflated: The deflated matrix.
    """
    return A - eigenvalue * np.outer(eigenvector, eigenvector)

def find_all_eigenvalues_eigenvectors(A, num_eigenvalues=None):
    """
    Find all eigenvalues and eigenvectors of a square matrix A using power iteration and deflation.
    
    Parameters:
    - A: The input square matrix.
    - num_eigenvalues: Number of eigenvalues to find (default is the size of the matrix).
    
    Returns:
    - eigenvalues: List of all eigenvalues.
    - eigenvectors: List of all eigenvectors.
    """
    n = A.shape[0] if num_eigenvalues is None else num_eigenvalues
    eigenvalues = []
    eigenvectors = []
    
    A_copy = A.copy()  # Copy of the matrix to apply deflation
    
    for _ in range(n):
        # Find the dominant eigenvalue and eigenvector using power iteration
        eigenvalue, eigenvector = power_iteration(A_copy)
        eigenvalues.append(eigenvalue)
        eigenvectors.append(eigenvector)
        
        # Deflate the matrix to remove the found eigenpair's influence
        A_copy = deflate_matrix(A_copy, eigenvalue, eigenvector)
    
    return np.array(eigenvalues), np.array(eigenvectors)

# Example matrix
A = np.array([[4, 2],
              [1, 3]])

# Find all eigenvalues and eigenvectors using power iteration and deflation
eigenvalues, eigenvectors = find_all_eigenvalues_eigenvectors(A)
print("Eigenvalues:", eigenvalues)
print("Eigenvectors:\n", eigenvectors)

# Compare with numpy's built-in function for reference
eigenvalues_np, eigenvectors_np = np.linalg.eig(A)
print("\nNumpy Eigenvalues:", eigenvalues_np)
print("Numpy Eigenvectors:\n", eigenvectors_np)


Eigenvalues: [5. 2.]
Eigenvectors:
 [[ 8.94427191e-01  4.47213596e-01]
 [-3.56788796e-10  1.00000000e+00]]

Numpy Eigenvalues: [5. 2.]
Numpy Eigenvectors:
 [[ 0.89442719 -0.70710678]
 [ 0.4472136   0.70710678]]


## Using np.linalg:

In [2]:
# Define a square matrix
matrix = np.array([[4, 2], [1, 3]])

# Perform Eigen decomposition
eigen_values, eigen_vectors = np.linalg.eig(matrix)

# Display the eigenvalues and eigenvectors
print("Eigenvalues:", eigen_values)
print("Eigenvectors:\n", eigen_vectors)

# Reconstruct the matrix from eigen decomposition
V = eigen_vectors
V_inverse = np.linalg.inv(V)
diag = np.diag(eigen_values)
reconstructed_matrix = V @ diag @ V_inverse

# Display the reconstructed matrix
print("\nReconstructed Matrix:\n", reconstructed_matrix)


Eigenvalues: [5. 2.]
Eigenvectors:
 [[ 0.89442719 -0.70710678]
 [ 0.4472136   0.70710678]]

Reconstructed Matrix:
 [[4. 2.]
 [1. 3.]]


# Part 2: Singular Value Decomposition (SVD) for Non-Square Matrices

**Singular Value Decomposition (SVD)** is a more general decomposition method that can be applied to any matrix, regardless of whether it is square or non-square. SVD breaks down a matrix into three other matrices, providing a deeper understanding of its structure.

### Explanation:

For any matrix $A$ of dimensions $m \times n$, SVD represents $A$ as the product of three matrices:

$$ A = U \times \Sigma \times V^T $$

Where:
- $U$ is an $m \times m$ matrix whose columns are the **left singular vectors** of $A$.
- $\Sigma$ is an $m \times n$ diagonal matrix containing the **singular values** of $A$.
- $V^T$ is the transpose of an $n \times n$ matrix whose columns are the **right singular vectors** of $A$.

The singular values in $\Sigma$ are non-negative and sorted in decreasing order. They represent the "strength" or "importance" of the corresponding singular vectors.

### Proof (Sketch):

SVD can be derived from the **Eigen Decomposition** of $A^T A$. For a matrix $A$, the matrix $A^T A$ is square and symmetric, meaning it has an Eigen Decomposition:

$$ A^T A = V \times \Lambda \times V^T $$

The eigenvalues of $A^T A$ are the squares of the singular values of $A$, and the eigenvectors of $A^T A$ are the right singular vectors of $A$.

Similarly, the matrix $A A^T$ can be decomposed to obtain the left singular vectors of $A$.

This approach allows us to decompose any matrix, regardless of its shape.


## From scratch:

In [3]:
import numpy as np

def svd_from_scratch(A, tol=1e-9):
    """
    Perform Singular Value Decomposition (SVD) from scratch.

    Parameters:
    - A: The input matrix.
    - tol: Tolerance for handling small values that should be treated as zero.

    Returns:
    - U: Matrix of left singular vectors.
    - Sigma: Diagonal matrix of singular values.
    - Vt: Transpose of the matrix of right singular vectors.
    """
    # Step 1: Compute A^T * A and A * A^T
    AtA = np.dot(A.T, A)
    AAt = np.dot(A, A.T)

    # Step 2: Compute eigenvalues and eigenvectors of A^T * A (V and Sigma)
    eigenvalues_AtA, V = np.linalg.eig(AtA)
    singular_values = np.sqrt(np.abs(eigenvalues_AtA))
    
    # Sort singular values and corresponding right singular vectors (V)
    sorted_indices = np.argsort(-singular_values)
    singular_values = singular_values[sorted_indices]
    V = V[:, sorted_indices]

    # Step 3: Compute left singular vectors U from A * A^T using singular values
    U = np.zeros(AAt.shape)
    for i in range(len(singular_values)):
        if singular_values[i] > tol:
            U[:, i] = np.dot(A, V[:, i]) / singular_values[i]
    
    # Step 4: Construct the Sigma matrix with singular values on the diagonal
    Sigma = np.zeros(A.shape)
    np.fill_diagonal(Sigma, singular_values)
    
    return U, Sigma, V.T

# Example matrix
A = np.array([[3, 1, 1],
              [-1, 3, 1]])

# Perform SVD from scratch
U, Sigma, Vt = svd_from_scratch(A)

print("U matrix:\n", U)
print("Sigma matrix:\n", Sigma)
print("V transpose matrix:\n", Vt)

# Reconstruct the matrix from SVD
reconstructed_matrix = np.dot(U, np.dot(Sigma, Vt))
print("\nReconstructed Matrix:\n", reconstructed_matrix)


U matrix:
 [[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
Sigma matrix:
 [[3.46410162 0.         0.        ]
 [0.         3.16227766 0.        ]]
V transpose matrix:
 [[-4.08248290e-01 -8.16496581e-01 -4.08248290e-01]
 [-8.94427191e-01  4.47213595e-01  5.07704275e-16]
 [ 1.82574186e-01  3.65148372e-01 -9.12870929e-01]]

Reconstructed Matrix:
 [[ 3.  1.  1.]
 [-1.  3.  1.]]


## Using np.linalg:

In [4]:
# Define a non-square matrix
matrix = np.array([[3, 1, 1], [-1, 3, 1]])

# Perform SVD
U, sigma, Vt = np.linalg.svd(matrix)

# Display the results
print("U matrix:\n", U)
print("Sigma values:", sigma)
print("V transpose matrix:\n", Vt)

# Reconstruct the matrix from SVD
Sigma = np.zeros(matrix.shape)
Sigma[:len(sigma), :len(sigma)] = np.diag(sigma)
reconstructed_matrix = U @ Sigma @ Vt

# Display the reconstructed matrix
print("\nReconstructed Matrix:\n", reconstructed_matrix)


U matrix:
 [[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
Sigma values: [3.46410162 3.16227766]
V transpose matrix:
 [[-4.08248290e-01 -8.16496581e-01 -4.08248290e-01]
 [-8.94427191e-01  4.47213595e-01  5.26260748e-16]
 [-1.82574186e-01 -3.65148372e-01  9.12870929e-01]]

Reconstructed Matrix:
 [[ 3.  1.  1.]
 [-1.  3.  1.]]


## Reconstructing Matrices from Decompositions

Matrix decomposition allows us to break down matrices into simpler components, but it's essential to verify that we can reconstruct the original matrices from these components. 

For both Eigen Decomposition and SVD, we can reconstruct the original matrix by multiplying the decomposed matrices. This step is crucial to ensure the correctness of the decomposition process. The reconstructed matrices should closely match the original matrices, validating that the decompositions were performed accurately.

In the examples provided above, we demonstrated how to reconstruct the matrices from both Eigen Decomposition and SVD. The results show that the reconstructed matrices are nearly identical to the original ones.


# Practical Applications of Matrix Decomposition

Matrix decomposition techniques are not just theoretical; they have numerous real-world applications in various fields, including:

- **Principal Component Analysis (PCA)**: Eigen decomposition is a key component of PCA, which is widely used for dimensionality reduction in machine learning and data analysis. By decomposing a covariance matrix into its eigenvectors, PCA identifies the directions (principal components) along which the data varies the most.
  
- **Latent Semantic Analysis (LSA)**: SVD is used in natural language processing to discover latent relationships between words and documents. By decomposing a term-document matrix using SVD, LSA can identify hidden topics in a corpus of text.

- **Image Compression**: SVD is employed in image processing to compress images by retaining only the most significant singular values. By discarding the less important singular values, we can reduce the size of the image data while preserving essential features.



---
## Thank You for Exploring This Notebook!


If you have any questions, suggestions, or just want to discuss any of the topics further, please don't hesitate to reach out or leave a comment. Your feedback is not only welcome but also invaluable!

Happy analyzing, and stay curious!

---